### clustering

In [6]:
import matplotlib.pyplot as plt
import warnings
import networkx as nx
from shapely.geometry import Point
import random
from scipy.spatial import ConvexHull, convex_hull_plot_2d
#import matplotlib.cm as cm

### to switch the warnings off
#from shapely.errors import ShapelyDeprecationWarning
#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings('ignore', "", UserWarning)
#warnings.filterwarnings('ignore', "", ShapelyDeprecationWarning)
#to supress all warnings: 
#warnings.filterwarnings("ignore")

import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gp
from collections import defaultdict 

def cloud_cluster(bus_stops_df, capacity, distances_to_arena_check, bus_names_check):
    cluster_nodelist_dict = defaultdict(list)
    cluster_number = 0  # cluster number
    while len(bus_names_check) != 0:#iterate till all nodes are assigned to clusters
        people_assigned = 0
        max_dist = max(distances_to_arena_check)
        ind_max_dist = distances_to_arena_check.index(max_dist)
        cluster_center = bus_names_check[ind_max_dist]
        distances_to_arena_check.remove(max_dist)
        bus_names_check.remove(cluster_center)
        #to ignore stops with 0 people assigned:
        people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
        if people_assigned_next == 0:
            continue
        cluster_nodelist_dict[cluster_number] += [cluster_center]  # assign cluster center to cluster
        # assigning people to cluster to check the capacity constraint:
        people_assigned += people_assigned_next
        # creating a list to assign to the current cluster:
        cluster_candidates = (matrix.loc[cluster_center,bus_names_check]).to_dict() 
        cluster_candidates_names = list(cluster_candidates.keys())
        cluster_candidates_dist = list(cluster_candidates.values())
        while len(cluster_candidates_dist) != 0:
            min_dist = min(cluster_candidates_dist)
            ind_min_dist = cluster_candidates_dist.index(min_dist)
            new_clust_elem = cluster_candidates_names[ind_min_dist]
            ind_bus_stop = bus_names_check.index(new_clust_elem)
            people_assigned_next = bus_stops_df.loc[new_clust_elem, 'passengers']
            # make sure we do not put in the cluster the stops with 0 people assigned
            if people_assigned_next == 0: #make sure we do not put in cluster the 
                distances_to_arena_check.pop(ind_bus_stop)
                bus_names_check.remove(new_clust_elem)
                cluster_candidates_dist.pop(ind_min_dist)
                cluster_candidates_names.remove(new_clust_elem)
                continue
            if people_assigned + people_assigned_next > capacity: #check capacity constraint
                break
            cluster_nodelist_dict[cluster_number] +=  [new_clust_elem]
            people_assigned += people_assigned_next
            distances_to_arena_check.pop(ind_bus_stop)
            bus_names_check.remove(new_clust_elem)
            cluster_candidates_dist.pop(ind_min_dist)
            cluster_candidates_names.remove(new_clust_elem) 
        cluster_number += 1
    return cluster_nodelist_dict


#CLUSTERING ALGORITHM with SEQUENCE approach

def sequence_cluster(bus_stops_df, capacity, distances_to_arena_check, bus_names_check):
    cluster_nodelist_dict = defaultdict(list)
    cluster_number = 0  # cluster number
    while len(bus_names_check) != 0:
        people_assigned = 0
        max_dist = max(distances_to_arena_check)
        ind_max_dist = distances_to_arena_check.index(max_dist)
        cluster_center = bus_names_check[ind_max_dist]
        distances_to_arena_check.remove(max_dist)
        bus_names_check.remove(cluster_center)
        people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
        if people_assigned_next == 0:
            continue
        cluster_nodelist_dict[cluster_number] += [cluster_center]
        people_assigned += people_assigned_next
        while (people_assigned + people_assigned_next <= capacity) and len(bus_names_check) != 0:
            cluster_candidates = (matrix.loc[cluster_center,bus_names_check]).to_dict()
            cluster_candidates_names = list(cluster_candidates.keys())
            cluster_candidates_dist = list(cluster_candidates.values())
            min_dist = min(cluster_candidates_dist)
            ind_min_dist = cluster_candidates_dist.index(min_dist)
            cluster_center = cluster_candidates_names[ind_min_dist]
            people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
            if people_assigned_next == 0:
                ind_bus_stop = bus_names_check.index(cluster_center)
                distances_to_arena_check.pop(ind_bus_stop)
                bus_names_check.remove(cluster_center)
                continue
            if people_assigned + people_assigned_next > capacity:
                break
            cluster_nodelist_dict[cluster_number] +=  [cluster_center]
            ind_bus_stop = bus_names_check.index(cluster_center)
            distances_to_arena_check.pop(ind_bus_stop)
            bus_names_check.remove(cluster_center)
            people_assigned += people_assigned_next
        cluster_number += 1
    return cluster_nodelist_dict

#CLUSTERING ALGORITHM with A-star approach
#k is hyperparameter to take into account portion of the distance to Arena
def a_star_cluster(bus_stops_df, capacity, distances_to_arena_check, bus_names_check, k):
    cluster_nodelist_dict = defaultdict(list)
    cluster_number = 0  # cluster number
    while len(bus_names_check) != 0:
        people_assigned = 0
        max_dist = max(distances_to_arena_check)
        ind_max_dist = distances_to_arena_check.index(max_dist)
        cluster_center = bus_names_check[ind_max_dist]
        distances_to_arena_check.remove(max_dist)
        bus_names_check.remove(cluster_center)
        cluster_nodelist_dict[cluster_number] += [cluster_center]
        people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
        if people_assigned_next == 0:
            continue
        people_assigned += people_assigned_next
        people_assigned_next = 0
        while (people_assigned + people_assigned_next <= capacity) and len(bus_names_check) != 0:
            cluster_candidates = (matrix.loc[cluster_center, bus_names_check]).to_dict()
            for node, dist in cluster_candidates.items():
                dist_add = matrix.loc[node, "Schlachthof"]
                cluster_candidates[node] = dist + dist_add/k
            cluster_candidates_names = list(cluster_candidates.keys())
            cluster_candidates_dist = list(cluster_candidates.values())
            min_dist = min(cluster_candidates_dist)
            ind_min_dist = cluster_candidates_dist.index(min_dist)
            cluster_center = cluster_candidates_names[ind_min_dist]
            people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
            if people_assigned_next == 0:
                ind_bus_stop = bus_names_check.index(cluster_center)
                distances_to_arena_check.pop(ind_bus_stop)
                bus_names_check.remove(cluster_center)
                continue
            if people_assigned + people_assigned_next > capacity:
                break
            cluster_nodelist_dict[cluster_number] +=  [cluster_center]
            ind_bus_stop = bus_names_check.index(cluster_center)
            distances_to_arena_check.pop(ind_bus_stop)
            bus_names_check.remove(cluster_center)
            people_assigned += people_assigned_next
        cluster_number += 1
    return cluster_nodelist_dict

#CLUSTERING ALGORITHM with A-star k-next approach
#k is hyperparameter how many next closest points to compare to get the one that leads to Arena
def a_star_k_next_cluster(bus_stops_df, capacity, distances_to_arena_check, bus_names_check, k = 3):
    cluster_nodelist_dict = defaultdict(list)
    cluster_number = 0  # cluster number
    while len(bus_names_check) != 0:
        people_assigned = 0
        max_dist = max(distances_to_arena_check)
        ind_max_dist = distances_to_arena_check.index(max_dist)
        cluster_center = bus_names_check[ind_max_dist]
        distances_to_arena_check.remove(max_dist)
        bus_names_check.remove(cluster_center)
        cluster_nodelist_dict[cluster_number] += [cluster_center]
        people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
        if people_assigned_next == 0:
            continue
        people_assigned += people_assigned_next
        people_assigned_next = 0
        while (people_assigned + people_assigned_next <= capacity) and len(bus_names_check) != 0:
            cluster_candidates = (matrix.loc[cluster_center, bus_names_check]).to_dict()
            cluster_candidates_names = list(cluster_candidates.keys())
            cluster_candidates_dist = list(cluster_candidates.values())
            #Sorting lists to get access to the closest ti cluster center:
            lists_to_sort = zip(cluster_candidates_dist, cluster_candidates_names)
            sorted_lists = sorted(lists_to_sort)  # sorted according to cluster_candidates_dist
            cluster_candidates_dist_sorted = [elem for elem,_ in sorted_lists][:k]
            cluster_candidates_names_sorted = [elem for _,elem in sorted_lists][:k]
            #create new dictionary with k closest elements to the cluster center
            cluster_candidates_k_next = dict(zip(cluster_candidates_names_sorted, cluster_candidates_dist_sorted))
            #add the distance to the arena to evaluate the best way to go
            for node, dist in cluster_candidates_k_next.items():
                dist_add = matrix.loc[node, "Schlachthof"]
                cluster_candidates_k_next[node] = dist + dist_add*0.99 #0.99 to not choose next on the same line after closest
            cluster_candidates_k_next_dist = list(cluster_candidates_k_next.values())
            cluster_candidates_k_next_names = list(cluster_candidates_k_next.keys())
            min_dist = min(cluster_candidates_k_next_dist)
            ind_min_dist = cluster_candidates_k_next_dist.index(min_dist)
            cluster_center = cluster_candidates_k_next_names[ind_min_dist]
            people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
            if people_assigned_next == 0:
                ind_bus_stop = bus_names_check.index(cluster_center)
                distances_to_arena_check.pop(ind_bus_stop)
                bus_names_check.remove(cluster_center)
                continue
            if people_assigned + people_assigned_next > capacity:
                break
            cluster_nodelist_dict[cluster_number] +=  [cluster_center]
            ind_bus_stop = bus_names_check.index(cluster_center)
            distances_to_arena_check.pop(ind_bus_stop)
            bus_names_check.remove(cluster_center)
            people_assigned += people_assigned_next
        cluster_number += 1
    return cluster_nodelist_dict

#CLUSTERING ALGORITHM with CONVEX HULL center assignment and CLOUD approach

def convex_cloud_cluster(bus_stops_df, capacity, distances_to_arena_check, bus_names_check, choice = 'random'):
    cluster_nodelist_dict = defaultdict(list)
    cluster_number = 0  # cluster number
    while len(bus_names_check) != 0:#iterate till all nodes are assigned to clusters
        people_assigned = 0
        #determine convex points:
        if len(bus_names_check) >= 3: #to calculate convex hull we need more than 3 points:
            points_unassigned = np.array([[bus_stops_df.loc[name]['x']] + [bus_stops_df.loc[name]['y']] for name in bus_names_check])
            hull = ConvexHull(points_unassigned)
            bus_names_to_choose = [bus_names_check[index] for index in hull.vertices]
            distances_to_arena_to_choose = [distances_to_arena_check[index] for index in hull.vertices]
        else: 
            bus_names_to_choose = bus_names_check
            distances_to_arena_to_choose = distances_to_arena_check
        #choose among convex point:
        #choose randomly:
        if choice == 'random':
            cluster_center = random.choice(bus_names_to_choose)
            ind_to_remove = bus_names_check.index(cluster_center)
            distances_to_arena_check.pop(ind_to_remove)
            bus_names_check.remove(cluster_center)
        if choice == 'distance':
            max_dist = max(distances_to_arena_to_choose)
            ind_max_dist = distances_to_arena_to_choose.index(max_dist)
            cluster_center = bus_names_to_choose[ind_max_dist]       
            distances_to_arena_check.remove(max_dist)
            bus_names_check.remove(cluster_center)       
        #to ignore stops with 0 people assigned:
        people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
        if people_assigned_next == 0:
            continue
        cluster_nodelist_dict[cluster_number] += [cluster_center]  # assign cluster center to cluster
        # assigning people to cluster to check the capacity constraint:
        people_assigned += people_assigned_next
        # creating a list to assign to the current cluster:
        cluster_candidates = (matrix.loc[cluster_center,bus_names_check]).to_dict() 
        cluster_candidates_names = list(cluster_candidates.keys())
        cluster_candidates_dist = list(cluster_candidates.values())
        while len(cluster_candidates_dist) != 0:
            min_dist = min(cluster_candidates_dist)
            ind_min_dist = cluster_candidates_dist.index(min_dist)
            new_clust_elem = cluster_candidates_names[ind_min_dist]
            ind_bus_stop = bus_names_check.index(new_clust_elem)
            people_assigned_next = bus_stops_df.loc[new_clust_elem, 'passengers']
            # make sure we do not put in the cluster the stops with 0 people assigned
            if people_assigned_next == 0: #make sure we do not put in cluster the 
                distances_to_arena_check.pop(ind_bus_stop)
                bus_names_check.remove(new_clust_elem)
                cluster_candidates_dist.pop(ind_min_dist)
                cluster_candidates_names.remove(new_clust_elem)
                continue
            if people_assigned + people_assigned_next > capacity: #check capacity constraint
                break
            cluster_nodelist_dict[cluster_number] +=  [new_clust_elem]
            people_assigned += people_assigned_next
            distances_to_arena_check.pop(ind_bus_stop)
            bus_names_check.remove(new_clust_elem)
            cluster_candidates_dist.pop(ind_min_dist)
            cluster_candidates_names.remove(new_clust_elem) 
        cluster_number += 1
    return cluster_nodelist_dict

#CLUSTERING ALGORITHM with CONVEX HULL center assignment and SEQUENCE approach

def convex_sequence_cluster(bus_stops_df, capacity, distances_to_arena_check, bus_names_check, choice = 'random'):
    cluster_nodelist_dict = defaultdict(list)
    cluster_number = 0  # cluster number
    while len(bus_names_check) != 0:#iterate till all nodes are assigned to clusters
        people_assigned = 0
        #determine convex points:
        if len(bus_names_check) >= 3: #to calculate convex hull we need more than 3 points:
            points_unassigned = np.array([[bus_stops_df.loc[name]['x']] + [bus_stops_df.loc[name]['y']] for name in bus_names_check])
            hull = ConvexHull(points_unassigned)
            bus_names_to_choose = [bus_names_check[index] for index in hull.vertices]
            distances_to_arena_to_choose = [distances_to_arena_check[index] for index in hull.vertices]
        else: 
            bus_names_to_choose = bus_names_check
            distances_to_arena_to_choose = distances_to_arena_check
        #choose among convex point:
        #choose randomly:
        if choice == 'random':
            cluster_center = random.choice(bus_names_to_choose)
            ind_to_remove = bus_names_check.index(cluster_center)
            distances_to_arena_check.pop(ind_to_remove)
            bus_names_check.remove(cluster_center)
        if choice == 'distance':
            max_dist = max(distances_to_arena_to_choose)
            ind_max_dist = distances_to_arena_to_choose.index(max_dist)
            cluster_center = bus_names_to_choose[ind_max_dist]       
            distances_to_arena_check.remove(max_dist)
            bus_names_check.remove(cluster_center)       
        #to ignore stops with 0 people assigned:
        people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
        if people_assigned_next == 0:
            continue
        cluster_nodelist_dict[cluster_number] += [cluster_center]  # assign cluster center to cluster
        # assigning people to cluster to check the capacity constraint:
        people_assigned += people_assigned_next
        # creating a list to assign to the current cluster:
        people_assigned_next = 0
        while (people_assigned + people_assigned_next <= capacity) and len(bus_names_check) != 0:
            cluster_candidates = (matrix.loc[cluster_center,bus_names_check]).to_dict()
            cluster_candidates_names = list(cluster_candidates.keys())
            cluster_candidates_dist = list(cluster_candidates.values())
            min_dist = min(cluster_candidates_dist)
            ind_min_dist = cluster_candidates_dist.index(min_dist)
            cluster_center = cluster_candidates_names[ind_min_dist]
            people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
            if people_assigned_next == 0:
                ind_bus_stop = bus_names_check.index(cluster_center)
                distances_to_arena_check.pop(ind_bus_stop)
                bus_names_check.remove(cluster_center)
                continue
            if people_assigned + people_assigned_next > capacity:
                break
            cluster_nodelist_dict[cluster_number] +=  [cluster_center]
            ind_bus_stop = bus_names_check.index(cluster_center)
            distances_to_arena_check.pop(ind_bus_stop)
            bus_names_check.remove(cluster_center)
            people_assigned += people_assigned_next
        cluster_number += 1
    return cluster_nodelist_dict

#CLUSTERING ALGORITHM with CONVEX HULL center assignment and A-star approach
def convex_a_star_cluster(bus_stops_df, capacity, distances_to_arena_check, bus_names_check, k = 3, choice = 'random'):
    cluster_nodelist_dict = defaultdict(list)
    cluster_number = 0  # cluster number
    while len(bus_names_check) != 0:#iterate till all nodes are assigned to clusters
        people_assigned = 0
        #determine convex points:
        if len(bus_names_check) >= 3: #to calculate convex hull we need more than 3 points:
            points_unassigned = np.array([[bus_stops_df.loc[name]['x']] + [bus_stops_df.loc[name]['y']] for name in bus_names_check])
            hull = ConvexHull(points_unassigned)
            bus_names_to_choose = [bus_names_check[index] for index in hull.vertices]
            distances_to_arena_to_choose = [distances_to_arena_check[index] for index in hull.vertices]
        else: 
            bus_names_to_choose = bus_names_check
            distances_to_arena_to_choose = distances_to_arena_check
        #choose among convex point:
        #choose randomly:
        if choice == 'random':
            cluster_center = random.choice(bus_names_to_choose)
            ind_to_remove = bus_names_check.index(cluster_center)
            distances_to_arena_check.pop(ind_to_remove)
            bus_names_check.remove(cluster_center)
        if choice == 'distance':
            max_dist = max(distances_to_arena_to_choose)
            ind_max_dist = distances_to_arena_to_choose.index(max_dist)
            cluster_center = bus_names_to_choose[ind_max_dist]       
            distances_to_arena_check.remove(max_dist)
            bus_names_check.remove(cluster_center)       
        #to ignore stops with 0 people assigned:
        people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
        if people_assigned_next == 0:
            continue
        cluster_nodelist_dict[cluster_number] += [cluster_center]  # assign cluster center to cluster
        # assigning people to cluster to check the capacity constraint:
        people_assigned += people_assigned_next
        # creating a list to assign to the current cluster:
        people_assigned_next = 0
        while (people_assigned + people_assigned_next <= capacity) and len(bus_names_check) != 0:
            cluster_candidates = (matrix.loc[cluster_center, bus_names_check]).to_dict()
            cluster_candidates_names = list(cluster_candidates.keys())
            cluster_candidates_dist = list(cluster_candidates.values())
            #Sorting lists to get access to the closest to cluster center:
            lists_to_sort = zip(cluster_candidates_dist, cluster_candidates_names)
            sorted_lists = sorted(lists_to_sort)  # sorted according to cluster_candidates_dist
            cluster_candidates_dist_sorted = [elem for elem,_ in sorted_lists][:k]
            cluster_candidates_names_sorted = [elem for _,elem in sorted_lists][:k]
            #create new dictionary with k closest elements to the cluster center
            cluster_candidates_k_next = dict(zip(cluster_candidates_names_sorted, cluster_candidates_dist_sorted))
            #add the distance to the arena to evaluate the best way to go
            for node, dist in cluster_candidates_k_next.items():
                dist_add = matrix.loc[node, "Schlachthof"]
                cluster_candidates_k_next[node] = dist + dist_add*0.99 #0.999 to not choose next on the same line after closest
            cluster_candidates_k_next_dist = list(cluster_candidates_k_next.values())
            cluster_candidates_k_next_names = list(cluster_candidates_k_next.keys())
            min_dist = min(cluster_candidates_k_next_dist)
            ind_min_dist = cluster_candidates_k_next_dist.index(min_dist)
            cluster_center = cluster_candidates_k_next_names[ind_min_dist]
            people_assigned_next = bus_stops_df.loc[cluster_center, 'passengers']
            if people_assigned_next == 0:
                ind_bus_stop = bus_names_check.index(cluster_center)
                distances_to_arena_check.pop(ind_bus_stop)
                bus_names_check.remove(cluster_center)
                continue
            if people_assigned + people_assigned_next > capacity:
                break
            cluster_nodelist_dict[cluster_number] +=  [cluster_center]
            ind_bus_stop = bus_names_check.index(cluster_center)
            distances_to_arena_check.pop(ind_bus_stop)
            bus_names_check.remove(cluster_center)
            people_assigned += people_assigned_next
        cluster_number += 1
    return cluster_nodelist_dict


def run_CL(scenario,method):
    if scenario=='1':
        #read data from scenario 1
        bus_stops_people=pd.read_csv('scenario_1_LK.csv',sep=",")         
    if scenario=='2':
        #read data from scenario 2
        bus_stops_people=pd.read_csv('scenario_2_LK.csv',sep=",")
    if scenario=='3':
        #read data from scenario 3
        bus_stops_people=pd.read_csv('scenario_3_LK.csv',sep=",")
        
    #uploading matrix with distances among stations
    osminfo = pd.read_csv("stop_info_osm.csv",sep=",")
    # merge with osm information
    bus_stops_people = (osminfo.merge(bus_stops_people,
                          on=['name'], how='left',
                          indicator=True).drop(columns='_merge'))
    #read data from full matrix LK
    matrix = pd.read_csv('full_distance_matrix_LK.csv', sep=",", index_col = 'name')
    
    #list of distances to arena:
    distances_to_arena = list(matrix.loc["Schlachthof"])

    #list of bus stop names
    bus_names = list(matrix.columns)

    #creating the dataframe with bus_stops and people assigned to them:
    bus_stops_df = bus_stops_people
    bus_stops_df.index = bus_stops_df['name']
    bus_stops_df= bus_stops_df.drop('name', axis = 1)

    #CLUSTERING ALGORITHM with CLOUD approach

    distances_to_arena_check = distances_to_arena.copy() #list of possible 
    bus_names_check = bus_names.copy()
    capacity = 70 #how many people we can get from one cluster
    
    if method=='convex_hull_seq_deter':
        #CONVEX_HULL_SEQUENCE:
        cluster_nodelist_dict = convex_sequence_cluster(bus_stops_df, capacity, distances_to_arena_check, bus_names_check, choice = 'distance')
    if method=='convex_hull_cloud_deter':
        cluster_nodelist_dict = convex_cloud_cluster(bus_stops_df, capacity, distances_to_arena_check, bus_names_check, choice = 'distance')
     
    return cluster_nodelist_dict

### SA

In [1]:
import time
import pandas as pd
import numpy as np

# DATA PRE-PROCESSING

#check if a certain stop contained in a cluster
def stInLst(lst,stop):
    for st in lst:
        if st==stop:
            return True
    return False

#check if a stop appeared in a list of clusters
def checkStop(lstoflst,stop):
    i=0
    for sublst in lstoflst:
        i+=1
        if stInLst(sublst,stop):
            print('sublist nr ',i)
            return True
    return False           
def remove_arena(lst,arena):
    lst_remove=lst[:]
    for sublst in lst_remove:
        if stInLst(sublst,arena)==True:
            sublst.remove(arena)
    return lst_remove

def add_arena(lst,arena):
    lst_add=lst[:]
    for sublst in lst_add:
        sublst.append(arena)
    return lst_add

#input: lst_of_cluster with stops with non-zero passengers

def pre_processing(lst_of_clusters,arena):
    lst_of_clusters_remove=remove_arena(lst_of_clusters,arena)
    lst_of_clusters_process=add_arena(lst_of_clusters,arena)
    return lst_of_clusters_process

#--------------------------------------------------------------------------------------


# SA ALGORITHM

# function for SA

# define initial temperature for a cluster
def tmax(cluster):
    T=[]
    for st1 in cluster:
        for st2 in cluster:
            T.append(df.loc[st1,st2])
    return max(T)

#two-opt neighborhood operator
# s is a list of stops
def two_opt(s,i,j):
    s_prime=[]
    if i==0:
        t1=[]
    else:
        t1=s[0:i]
    t2=s[i:(j+1)]
    t2.reverse()
    t3=s[(j+1)::]
    s_prime=t1+t2+t3
    return s_prime

#cost function: distance, lst is a list of clusters
def cost(lst):
    l = 0
    for i in range(len(lst)-1):
        l += df.loc[lst[i]][lst[i+1]]
    #l += df.loc[lst[len(lst)-1],stop] 
    return l

#get the optimal route from a list of two routes
#list_check=[(['a'],1),(['b'],2)]
def opt_sol(list_check):
    if list_check[0][1]>list_check[1][1]:
        return list_check[1]
    else:
        return list_check[0]
    
#lst is list of clusters
def shortPath_SA_stops(lst,P,Tmax, alpha):
    best_sol= [] #save the good iterations with improvement (does not take no_change into account if it does not improve)
    k_no_change=0
    s = lst
    c = cost(lst)
    ntrial = 1
    #T = 35 # initial 30
    T=Tmax
    #alpha = 0.9
    while (ntrial <= N) and (T> T0) and (k_no_change<K):   
        for i in range(0,len(lst)-2):# dummy depots
            for j in range(i+1,len(lst)-1):
                s1=two_opt(s,i,j)
                c1 = cost(s1)
                if c1 < c:
                    s, c = s1, c1
                    best_sol.append((s1,c1))
                else:
                    if np.exp(-(c1 - c)/T) > P:
                        s, c = s1, c1
                        k_no_change=0
                    else:
                        k_no_change+=1
        T = alpha*T
        ntrial += 1
    if len(best_sol)!=0:
        lst_opt=[best_sol[-1], (s,c)]
        result=opt_sol(lst_opt)
    else:
        result=s,c
    return result

def run_SA(lst_clusters,P,alpha,K,T0,N):
    start = time.time()
    routes=[]
    total_route=0
    for sublst in lst:
        Tmax=tmax(sublst)
        result=shortPath_SA_stops(sublst,P,Tmax,alpha)
        total_route+=result[-1]
        routes.append(result)
    end=time.time()
    duration=end-start
    return total_route, routes, duration


### Run clustering

In [19]:
# run CL 
scenario='2'
method='convex_hull_seq_deter'

cluster_nodelist_dict=run_CL(scenario,method)

number_of_clusters = len(cluster_nodelist_dict.keys())#amount of clusters

#to calculate amountt of bus_stops in all clusters
amount = 0
len_list = []
for list_ in list(cluster_nodelist_dict.values()):
    amount += len(list_)


#to check repeated nodes in clusters:
amount = 0
len_list = []
list_of_lists = list(cluster_nodelist_dict.values())
alarm = []
for list_ in list_of_lists:
    list_of_lists_check = list_of_lists.copy()
    list_of_lists_check.remove(list_)
    count = 0
    for elem in list_:
        for list_1 in list_of_lists_check:
            if elem in list_1:
                count += 1
    alarm.append(count)

### run SA

In [20]:
lst_of_clusters=[]
for i in cluster_nodelist_dict.keys():
    lst_of_clusters.append(cluster_nodelist_dict[i])

arena='Schlachthof'
#full distance matrix df
df=matrix
# run Data Pre-processing
lst_of_clusters_process=pre_processing(lst_of_clusters,arena)
lst=lst_of_clusters_process[:]

# RUNNING SA
P=0.5
N=500
T0=0.1
K=200
alpha=0.95
result_run=run_SA(lst,P,alpha,K,T0,N)
print(result_run[0], result_run[2])

803156.1550000003 460.7015235424042
